In [2]:
import sys
import os
import os.path

import numpy as np
import skimage.io
import glob

import shutil
#from config import config_vars

from tqdm import tqdm

import matplotlib.pyplot as plt
import skimage.morphology

import tensorflow as tf
import keras

import utils.metrics
import utils.model_builder
from skimage.transform import resize
from skimage import measure
from skimage import filters

Using TensorFlow backend.


#### Load images 

In [3]:
filename = sorted(os.listdir('/home/jupyter/cell-segmentation/aitslab_boundary_labels/resized_images/'))
image_names = [os.path.join('/home/jupyter/cell-segmentation/resized_aitslab_png_images/', f) for f in filename]
#image_names.remove('/home/jupyter/aitslab_images/norm_images/.ipynb_checkpoints')

In [4]:
imagebuffer = skimage.io.imread_collection(image_names)
images = imagebuffer.concatenate()

#### Make predictions with Otsus and save  

In [5]:
config_vars = {
    'boundary_size': 2
}

In [13]:
annotation_dir = "/home/jupyter/cell-segmentation/otsus_prediction/annotation/"
labels_dir = "/home/jupyter/cell-segmentation/otsus_prediction/labels/"


for n,i in enumerate(images):
    thresh_val = skimage.filters.threshold_otsu(i) #find threshold using otsus method
    mask = i > thresh_val # create mask 
    object_labels = skimage.measure.label(mask) # convert to skimage label
    annot = skimage.morphology.label(object_labels) # convert to annotation
    
    
    #skimage.io.imsave(annotation_dir + filename[n], annot)
    
    #annot = skimage.morphology.remove_small_objects(annot, min_size=config_vars["min_nucleus_size"]) #unclear if necessary
    boundaries = skimage.segmentation.find_boundaries(annot) #extract boundaries
    
    #plt.imshow(annot)
    #break
    
    for k in range(2, config_vars["boundary_size"], 2):
        boundaries = skimage.morphology.binary_dilation(boundaries)
    
    # prepare buffer for binary label
    label_binary = np.zeros((annot.shape + (3,)))

    # write binary label
    label_binary[(annot == 0) & (boundaries == 0), 0] = 1
    label_binary[(annot != 0) & (boundaries == 0), 1] = 1
    label_binary[boundaries == 1, 2] = 1
    
    
    print(label_binary)
    break
    
    
    
    #plt.imshow(label_binary)
    #break
    
    # save it - converts image to range from 0 to 255
    skimage.io.imsave(labels_dir + filename[n], label_binary)

[[[1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  ...
  [1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  ...
  [1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  ...
  [1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 ...

 [[1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  ...
  [1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  ...
  [1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]
  ...
  [1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]]
